In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import pulp

## Load data

In [3]:
supply_data = pd.read_csv('./supply_data.csv', index_col=[0])
ship_data = pd.read_csv('./ship_data.csv', index_col=[0,1])

In [4]:
supply_data
ship_data

,supply
node,
PHX,700
AUS,200
GNV,200
LAX,-200
DFW,-300
ORD,-200
ATL,-150
JFK,-250


capacity  cost
node_orig node_dest                
PHX       ORD             200     6
          ATL             200     7
          DFW             200     3
          LAX             200     3
AUS       LAX             200     7
          DFW             200     2
          ATL             200     5
GNV       DFW             200     6
          ATL             200     4
          JFK             200     7
DFW       LAX             200     5
          ORD             200     4
          JFK             200     6
          ATL             200     2
ATL       JFK             200     5
          ORD             200     4
          DFW             200     2

## Build Optimization Model
Sets, Parameters, and Variables

In [5]:
# Sets 
N = supply_data.index
O = ship_data.index.levels[0]
D = ship_data.index.levels[1]
OD = ship_data.index
OD

MultiIndex([('PHX', 'ORD'),
            ('PHX', 'ATL'),
            ('PHX', 'DFW'),
            ('PHX', 'LAX'),
            ('AUS', 'LAX'),
            ('AUS', 'DFW'),
            ('AUS', 'ATL'),
            ('GNV', 'DFW'),
            ('GNV', 'ATL'),
            ('GNV', 'JFK'),
            ('DFW', 'LAX'),
            ('DFW', 'ORD'),
            ('DFW', 'JFK'),
            ('DFW', 'ATL'),
            ('ATL', 'JFK'),
            ('ATL', 'ORD'),
            ('ATL', 'DFW')],
           names=['node_orig', 'node_dest'])

In [6]:
# Parameters
supply = supply_data.loc[:,'supply']
cost = ship_data.loc[:,'cost']
capacity = ship_data.loc[:,'capacity']

In [7]:
# Variables
x = pulp.LpVariable.dicts('x',[(o,d) for o,d in OD], lowBound=0,cat='Continuous')

In [8]:
x

{('PHX', 'ORD'): x_('PHX',_'ORD'),
 ('PHX', 'ATL'): x_('PHX',_'ATL'),
 ('PHX', 'DFW'): x_('PHX',_'DFW'),
 ('PHX', 'LAX'): x_('PHX',_'LAX'),
 ('AUS', 'LAX'): x_('AUS',_'LAX'),
 ('AUS', 'DFW'): x_('AUS',_'DFW'),
 ('AUS', 'ATL'): x_('AUS',_'ATL'),
 ('GNV', 'DFW'): x_('GNV',_'DFW'),
 ('GNV', 'ATL'): x_('GNV',_'ATL'),
 ('GNV', 'JFK'): x_('GNV',_'JFK'),
 ('DFW', 'LAX'): x_('DFW',_'LAX'),
 ('DFW', 'ORD'): x_('DFW',_'ORD'),
 ('DFW', 'JFK'): x_('DFW',_'JFK'),
 ('DFW', 'ATL'): x_('DFW',_'ATL'),
 ('ATL', 'JFK'): x_('ATL',_'JFK'),
 ('ATL', 'ORD'): x_('ATL',_'ORD'),
 ('ATL', 'DFW'): x_('ATL',_'DFW')}

## Model

In [9]:
# Define the model

model = pulp.LpProblem('Cost minimizing network flow problem', pulp.LpMinimize)

In [10]:
# Define objective
model += pulp.lpSum(
    [x[o,d] * cost[o,d] for o,d in OD]
)

In [11]:
# Add supply/demand constraint at every node
for n in N:
    model += pulp.lpSum(x.get((n,d),0) for d in D) - pulp.lpSum(x.get((o,n),0) for o in O) == supply[n]

In [12]:
# Add edge capacity constraint
for o,d in OD:
    model += x.get((o,d),0) <= capacity[o,d]

In [13]:
# Run model
model.solve()
pulp.LpStatus[model.status]

1

'Optimal'

## Results

In [14]:
# Minimum cost
print(pulp.value(model.objective))

5300.0


In [15]:
# Optimal shipment selections
for v in model.variables():
    print(v.name, '=', v.varValue)

x_('ATL',_'DFW') = 0.0
x_('ATL',_'JFK') = 0.0
x_('ATL',_'ORD') = 0.0
x_('AUS',_'ATL') = 0.0
x_('AUS',_'DFW') = 200.0
x_('AUS',_'LAX') = 0.0
x_('DFW',_'ATL') = 50.0
x_('DFW',_'JFK') = 50.0
x_('DFW',_'LAX') = 0.0
x_('DFW',_'ORD') = 0.0
x_('GNV',_'ATL') = 0.0
x_('GNV',_'DFW') = 0.0
x_('GNV',_'JFK') = 200.0
x_('PHX',_'ATL') = 100.0
x_('PHX',_'DFW') = 200.0
x_('PHX',_'LAX') = 200.0
x_('PHX',_'ORD') = 200.0
